<a href="https://colab.research.google.com/github/voldrek/Meus_projetos/blob/main/RomaneioVersionDefinitiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re,os,openpyxl,chardet
import numpy as np
from openpyxl import load_workbook

# Pasta onde os arquivos CSV estão localizados
pasta_arquivos = 'teste'

# Lista para armazenar os DataFrames de cada arquivo CSV
dataframes = []

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        result = chardet.detect(file.read())
    return result['encoding']





# Iterar pelos arquivos na pasta
for arquivo in os.listdir(pasta_arquivos):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(pasta_arquivos, arquivo)
        df = pd.read_csv(caminho_arquivo, sep='\t', encoding='utf-16')
        dataframes.append(df)

# Concatenar os DataFrames em um único DataFrame
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Salvar o DataFrame concatenado em um novo arquivo CSV
arquivo_concatenado = 'concatenado.csv'
df_concatenado.to_csv(arquivo_concatenado, index=False, sep='\t', encoding='utf-16')

file_path = arquivo_concatenado
encoding = detect_encoding(file_path)
# Ler os dados do arquivo CSV
df = pd.read_csv(arquivo_concatenado, sep="\t", encoding=encoding)

wb = openpyxl.Workbook()
ws = wb.active

#Trasnforma a coluna do Service Id em string
df['Service Id'] = df['Service Id'].astype(str).apply(lambda x: x.replace('.', '') if not pd.isnull(x) else x)

# Variáveis para padrões de números para extração
padrao_celular = r'(\b\d{11}\b)'
padrao_iccd = r'\b\d{20}\b'
padrao_imei = r'(\b\d{15}\b)'
padrao_nf = r'(\b\d{8}\b)'


resultado = df[df['Service Id'].str.contains(padrao_imei, na=False)]
produtos_encontrados = resultado['Produto']
df['Aparelhos'] = produtos_encontrados

# Iterar sobre as linhas do DataFrame
for index, row in df.iterrows():
    service_data = row['Service Id']
    telefone = re.search(padrao_celular, service_data)
    iccd = re.search(padrao_iccd, service_data)
    imei = re.search(padrao_imei, service_data)
    if telefone:
        df.at[index, 'Numero telefone'] = telefone.group(0)
    if iccd:
      if 'ICCD' in df.columns:
            df.at[index, 'ICCD'] = iccd.group(0)
      else:
            df['ICCD'] = None

    if imei:
        if 'IMEI' in df.columns:
            df.at[index, 'IMEI'] = imei.group(0)
        else:
            df['IMEI'] = None


#Salva os arquivos na nova pasta caso eles sejam localizados
with pd.ExcelWriter('resposta_romaneio.xlsx') as writer:
    df2 = pd.DataFrame()
    df2 = df2.astype(str)

    if 'ICCD' in df.columns:
      numero_iccd = df['ICCD'].dropna()
      numero_iccd.to_excel(writer, sheet_name='Planilha1', startcol=0, index=False)  # Coluna 'ICCD' na coluna A
    if 'Numero telefone' in df.columns:
      numero_telefone = df['Numero telefone'].dropna()
      numero_telefone.to_excel(writer, sheet_name='Planilha1', startcol=1, index=False)  # Coluna 'Telefone' na coluna b
    if 'Ordem de vendas' in df.columns:
      numero_ordem = df['Ordem de vendas'].dropna()
      numero_ordem.to_excel(writer, sheet_name='Planilha1', startcol=2, index=False)  # Coluna 'Ordem de Venda' na coluna c
    if 'Nota Fiscal' in df.columns:
      numero_nf = df['Nota Fiscal'].dropna()
      numero_nf.to_excel(writer, sheet_name='Planilha1', startcol=3, index=False) # Coluna 'Nota Fiscal' na coluna c
    if 'IMEI' in df.columns:
      numero_imei = df['IMEI'].dropna()
      numero_imei.to_excel(writer, sheet_name='Planilha1', startcol=4, index=False)  # Coluna 'IMEI' na coluna E
    if 'Conta de serviço' in df.columns:
      numero_conta = df['Conta de serviço'].dropna()
      numero_conta.to_excel(writer, sheet_name='Planilha1', startcol=5, index=False) #Coluna 'Conta de Serviço' na coluna F
    if 'Aparelhos' in df.columns:
      aparelhos = df['Aparelhos'].dropna()
      aparelhos.to_excel(writer,sheet_name='Planilha1', startcol=6, index=False) #Coluna 'Aparelhos' na coluna G



#Faz a limpeza dos Excluidos e novos
df2 = pd.read_excel('resposta_romaneio.xlsx')
df2 = df2.dropna(subset=['Numero telefone'])
colunas_inteiro = ['Numero telefone', 'Ordem de vendas', 'Nota Fiscal', 'IMEI', 'Conta de serviço']
for coluna in colunas_inteiro:
    df2[coluna] = df2[coluna].astype(float).apply(lambda x: '{:.0f}'.format(x) if not pd.isnull(x) else x)
df2.to_excel('resposta_romaneio.xlsx', index=False)

In [ ]:
df.info()

In [ ]:
df2.info()

In [47]:
df2.head(5)

,ICCD,Numero telefone,Ordem de vendas,Nota Fiscal,IMEI,Conta de serviço,Aparelhos
0,89551094444152093442,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
1,89551062369000249284,1.198513e+10,1.028472e+09,20315621.0,3.757860e+13,374674229,GESTAO_SMART_EMP_VOZ
2,89551094444152093277,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
3,89551062369000249276,1.198513e+10,1.028472e+09,20315621.0,3.757861e+13,374674229,GESTAO_SMART_EMP_VOZ
4,89551094444152092980,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
